
list wikimedia articles?


In [ ]:
const fs = require('fs');
const path = require('path')
//const zlib = require('zlib');
const { PassThrough, Readable } = require('stream');
const XmlStream = require('xml-stream');
const bz2 = require('unbzip2-stream');

const INDEX_FILE = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads', 'enwiki-20250420-pages-articles-multistream-index.txt')
const XML_FILE = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads', 'enwiki-20250420-pages-articles-multistream.xml.bz2')

async function readWikimedia(search) {

  // === Step 3: Tie it all together ===
  await new Promise(resolve => readIndex(search, ({ offset, length, titles }, finished) => {

    let isMatch = false
    if (search) {
      for (let i = 0; i < titles.length; i++) {
        if (titles[i].includes(search.toLocaleLowerCase())) {
          console.log('match found in set:', titles[i])
          isMatch = true
          break
        }
      }
    } else {
      isMatch = true
    }

    if (isMatch) {
      extractChunk(offset, offset + length, () => {



      });
    }

    if (finished) {
      resolve()
    }
  }));

}

function readIndex(search, callback) {
  const stream = fs.createReadStream(INDEX_FILE, {
    highWaterMark: 64 * 1024,
  });
  let lastOffset
  let leftover = ''
  let lastTitles = []
  stream.on('data', chunk => {
    const lines = (leftover + chunk.toString()).split('\n');
    leftover = lines.pop();
    for (const line of lines) {
      const [offsetStr, pageId, title] = line.split(':');
      const currentOffset = parseInt(offsetStr)
      if (currentOffset && currentOffset !== lastOffset) {
        if (lastOffset) {
          callback({ length: currentOffset - lastOffset, offset: lastOffset, titles: lastTitles });
        }
        lastOffset = currentOffset
        lastTitles = []
      }
      lastTitles.push(title.toLocaleLowerCase())
    }
  });

  stream.on('end', () => {
    callback({ length: fs.statSync(INDEX_FILE).size - lastOffset, offset: lastOffset, titles: lastTitles }, true);
  });
}

// Extract and parse one chunk
function extractChunk(startOffset, endOffset, callback) {
  console.log('reading:', startOffset, endOffset)
  const fileStream = fs.createReadStream(XML_FILE, {
    start: startOffset,
    end: endOffset - 1 // end is inclusive
  });

  const decompress = bz2();

  const wrapperStart = Readable.from(['<root>']);
  const wrapperEnd = Readable.from(['</root>']);

  const decompressedStream = fileStream.pipe(decompress);

  // Combine: <root> + decompressed + </root>
  const fullStream = PassThrough();
  wrapperStart.pipe(fullStream, { end: false });
  decompressedStream.pipe(fullStream, { end: false });
  decompressedStream.on('end', () => {
    wrapperEnd.pipe(fullStream);
  });

  const xml = new XmlStream(fullStream);

  xml.on('endElement: page', page => {
    //console.log(`Title: ${page.title}`);
  });

  xml.on('end', () => {
    console.log(`Chunk finished\n`);
    callback();
  });

  xml.on('error', err => {
    console.error('XML error:', err);
  });

  decompress.on('error', err => {
    console.error('BZ2 decompression error:', err);
  });
}

module.exports = readWikimedia
